# Word Power (Jupyter Notebook)
The purpose of this Jupyter Notebook was to provide a scratch space where we could rapidly prototype our code and experiment in order to make sure our code was working properly. It is possible to run this code in order to get results, but this code has been moved into a separate program that can leverage multiprocessing algorithms and should be much more performant. 

## System Requirements
Given that Jupyter Notebook/Python and Redis will need to load the data into system memory (RAM) the program can get very memory intense. It is necessary to at least have **16GB of RAM** in order to run the full program. In order to run this program, it is necessary to have Redis installed and working properly. If you are using Windows, Redis can be installed through [Chocolatey](https://chocolatey.org/) via `C:> choco install redis-64`. If you are running MacOSX you can install Redis via `$ brew install redis`. If you are running Linux or another Unix you should be able to install Redis through your package manager or compile from source.

## Dependencies
This program depends on having the necessary software and packages in order to run. First, you need to have `Python 3.5.2` installed. Next, you should be able to install all Python software dependencies through running `$ pip install -r requirements.txt`. In order to get the `lxml` package installed on Windows, it may be necessary to install the `.whl` file located in the `lib` project directory via `C:> pip install lib/lxml-3.6.4-cp35-cp35m-win_amd64.whl`. 

## Extensions
We load the Cython extension in order to gain a little more performance out of some of the Python code

In [1]:
%load_ext cython

c:\users\ajarrett3\appdata\local\programs\python\python35\lib\site-packages\Cython\Distutils\old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


## Imports
This is where all of the necessary packages/imports are added

In [2]:
from pandas import DataFrame, read_sas, read_csv
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

from SECEdgar.crawler import SecCrawler

from bs4 import BeautifulSoup as bs

import time
from datetime import datetime as dt
from datetime import date, timedelta

from collections import defaultdict

import os
import re
import lxml
import redis
import string
import pickle
import math
import zlib

import nltk
from nltk.corpus import stopwords

try: stopwords.words('english')
except LookupError: nltk.download('stopwords')
    
import statsmodels.api as sm
    
rds = redis.Redis()

## Load Data
This cell will check Redis in order to try to load data (CRSP and Compustat in addition to several dictionaries used later). The `crsp_comp.sas7bdat` file is generated from the `CRSP+Comp.sas` file which needs to be executed via SAS in order to get the necessary stock and accounting information. The `crsp_comp.sas7bdat` file needs to be placed in the `data/` folder of the project.

In [ ]:
# Read in SAS data set - takes a while so try to use redis...
try: data
except NameError:
    if rds.exists('data:word-power'):
        %time data = pickle.loads(zlib.decompress(rds.get('data:word-power')))
    else:
        %time data = read_sas("../data/crsp_comp.sas7bdat")

        # Trim the SAS data set
        data = data[['CUSIP','PERMNO','cik','tic','date','PRC','RET','vwretd']]

        # Sort the set by cusip, permno, cik, and then year (descending)
        data.sort_values(['CUSIP', 'PERMNO', 'cik', 'date'], ascending=[True, True, True, False], inplace=True)

        # Re-index the dataframe for better access
        data.reset_index(inplace=True)

        rds.set('data:word-power', zlib.compress(pickle.dumps(data)))

# We only need certain columns from the data set and we must set the right index for performance
try: df
except NameError:
    df = data[["cik", "date", "PRC", "RET", "vwretd"]]
    df.set_index(keys=['cik','date'], inplace=True)
    
# Positive words
try: pos_dict, pos_roots, pos_roots_map
except NameError:
    if rds.exists('data:pos-dict') and rds.exists('data:pos-roots') and rds.exists('data:pos-roots-map'):
        pos_dict = pickle.loads(zlib.decompress(rds.get('data:pos-dict')))
        pos_roots = pickle.loads(zlib.decompress(rds.get('data:pos-roots')))
        pos_roots_map = pickle.loads(zlib.decompress(rds.get('data:pos-roots-map')))
    else:
        # Read in the positive word list(s)
        pos_dict = read_csv("../data/pos_list.csv", header=None, names=['word'])
        pos_dict = set(pos_dict['word'])
        pos_roots = read_csv("../data/pos_roots.csv")
        pos_roots_map = dict(zip(list(pos_roots.word), list(pos_roots.group)))
        pos_roots = set(pos_roots['group'].drop_duplicates())

        # Save this data to redis for later
        rds.set('data:pos-dict', zlib.compress(pickle.dumps(pos_dict)))
        rds.set('data:pos-roots', zlib.compress(pickle.dumps(pos_roots)))
        rds.set('data:pos-roots-map', zlib.compress(pickle.dumps(pos_roots_map)))

# Negative words
try: neg_dict, neg_roots, neg_roots_map
except NameError:
    if rds.exists('data:neg-dict') and rds.exists('data:neg-roots') and rds.exists('data:neg-roots-map'):
        neg_dict = pickle.loads(zlib.decompress(rds.get('data:neg-dict')))
        neg_roots = pickle.loads(zlib.decompress(rds.get('data:neg-roots')))
        neg_roots_map = pickle.loads(zlib.decompress(rds.get('data:neg-roots-map')))
    else:
        # Read in the negative word list(s)
        neg_dict = read_csv("../data/neg_list.csv", header=None, names=['word'])
        neg_dict = set(neg_dict['word'])
        neg_roots = read_csv("../data/neg_roots.csv")
        neg_roots_map = dict(zip(list(neg_roots.word), list(neg_roots.group)))
        neg_roots = set(neg_roots['group'].drop_duplicates())

        # Save this data to redis for later
        rds.set('data:neg-dict', zlib.compress(pickle.dumps(neg_dict)))
        rds.set('data:neg-roots', zlib.compress(pickle.dumps(neg_roots)))
        rds.set('data:neg-roots-map', zlib.compress(pickle.dumps(neg_roots_map)))

# 2of12inf dictionary
try: dict_2of12inf
except NameError:
    if rds.exists('data:2of12inf'):
        dict_2of12inf = pickle.loads(zlib.decompress(rds.get('data:2of12inf')))
    else:
        # Read in the 2of12inf
        dict_2of12inf = read_csv("../data/2of12inf.txt", header=None, names=['word'])

        # Iterate through and remove the percent signs
        regex = re.compile(r'%$')
        dict_2of12inf.apply(lambda x: re.sub(regex, r'', x['word']), axis=1)
        dict_2of12inf = set(dict_2of12inf['word'])

        # Save this to redis for later
        rds.set('data:2of12inf', zlib.compress(pickle.dumps(dict_2of12inf)))

## 10-K Scraping from SEC Edgar
This cell will initiate the 10-K scraping algorithm using the `SECEdgar` library for Python. This will take a long time to run given that it downloads the past 20 files up until 2008. This library doesn't offer much customization and it would be better to implement our own scraper if we had enough time so we didn't have to download the number of years in multiples of 10. We would also be able to check if the file is a 10-K/A vs. a 10-K before we downloaded it as well which would save some time. We did have to make some modifications to this third-party package so it is included in the `lib/` folder of the project if the default package code does not work for you. The crawler will download the file to disk and place it in a folder format like ```"SEC-Edgar-data/" + tic + "/" + cik + "/" + file```. You will need about **50GB of free space** on your disk in order to download all of the files in the date range 1995 - 2008.

In [15]:
#%%capture

# Remove any duplicates where CUSIP, PERMNO, and CIK match
ciks = data.drop_duplicates(subset=['CUSIP', 'PERMNO', 'cik'])

# Only keep the cik and ticker column
ciks = ciks[['cik', 'tic']]

# Iterate over each CIK and pull the relevant 10k filings
crawler = SecCrawler()
end_date = '20081231'
count = '20'

for index, row in ciks.iterrows():
    cik = row.iloc[0]
    tic = row.iloc[1]
    crawler.filing_10K(tic, cik, end_date, count)


KeyError: "['tic'] not in index"

In [17]:
data['tic']

## Check Redis Function
This cell contains the `check_redis` function, which will check Redis in order to see if a given 10-K has been processed. When we later process a file, we will log this in Redis for easy checking in order to make sure all files are processed.

In [ ]:
%%cython
from cpython cimport bool

from __main__ import rds

def check_redis(str cleaned_key, str processed_key, str report_key):
    cdef bool processed = False
    cdef bool cleaned = False
    cdef str mtime
    
    if not rds.exists(cleaned_key):
        if not rds.exists(processed_key):
            # Temporary check to see if this file has been processed fully
            if rds.exists(report_key):
                mtime = rds.hget(report_key, 'mtime')

                if not rds.hexists(report_key, 'company_data'):
                    # Hasn't been cleaned with the new algorithm, so keep booleans False
                    pass
                elif rds.hexists(report_key, 'hist_ret'):
                    processed = True
                    cleaned = True

                    # Save to proper place in redis
                    rds.set(cleaned_key, mtime)
                    rds.set(processed_key, mtime)
                else:
                    cleaned = True

                    # Save to proper place in redis
                    rds.set(cleaned_key, mtime)
        else:
            processed = True
    else:
        # Check to see if this has really been cleaned (company_data exists)
        if rds.hexists(report_key, 'company_data'):
            cleaned = True
            if rds.exists(processed_key):
                processed = True
        
    return (cleaned, processed)

## Move File Function
This cell contains the `move_file` function that will move a file to a given folder when a certain condition is met. This condition can be that the file is outside of our date range (mainly because it is before 1995), it is an amended 10-K, we encountered an error trying to process the file, or we do not have any stock data for the company when the 10-K was filed.

In [ ]:
%%cython
import os

def move_file(fh, str fn, str folder, str tic, str cik, str filename, str message):
    # Generate the new name of the file
    cdef str s = os.sep
    cdef str new_name = 'data' + s + folder + s + tic + '-' + cik + '-' + filename

    # Close the file so that we can move it
    fh.close()
    os.rename(fn, new_name)
    print(message)

## Clean Function
This cell contains the `clean` function which is the workhorse function that loads a given 10-K file, finds the filing date, verifys we can process the file, strips out HTML and other formatting, and then filters out words that aren't in our dictionary. The resulting cleaned file is stored in Redis for later retrieval so this step doesn't have to be run again for a file once it has been cleaned.

In [ ]:
%%cython
import os
import re
import time
import string
import pickle
from datetime import datetime as dt
from lxml import etree
from io import StringIO

from bs4 import BeautifulSoup as bs

from __main__ import move_file, df, stopwords, rds, dict_2of12inf

from cpython cimport bool

regex = re.compile(r"(.*\.sgml\s+?:\s+?|.*\nFILED AS OF DATE:\s+?)([\d]+?)\n.*", re.S)

# This function handles the cleaning of the 10-K
def clean(str fn):
    cdef bool error = False
    
    cdef str s = os.sep
    cdef str tic = fn.split(s)[1]
    cdef str cik = fn.split(s)[2]
    cdef str filename = fn.split(s)[4]
    cdef str report_key = "report:" + cik + ":" + fn
    cdef str cleaned_key = "cleaned:" + cik + ":" + fn

    # Open the file, get all of the content, and then pull it into a parser
    fh = open(fn, 'r')
    cdef unicode contents = fh.read()

    # Clean up some of the text to fix malformed HTML before parsing it
    cdef list malformed_tags = ['ACCEPTANCE-DATETIME', 'TYPE', 'SEQUENCE', 'FILENAME', 'DESCRIPTION']
    cdef str tag
    for tag in malformed_tags:
        # Do a regex that replaces all of these malformed tags in the document
        regex = re.compile(r"(\n<%s>[^<]*?)\n" % re.escape(tag), re.I)
        contents = regex.sub(r"\1</%s>\n" % tag, contents)

    # Create the parser
    parser = etree.HTMLParser()
    document = etree.parse(StringIO(contents), parser)
    doc = document.getroot()
    
    # The document can either have a root node of sec-document or ims-document
    if doc.xpath('//sec-document') is not None:
        root = doc.xpath('//sec-document[1]')[0]
    elif doc.xpath('//ims-document') is not None: 
        root = doc.xpath('//ims-document[1]')[0]
    elif doc.xpath('//document') is not None:
        root = doc.xpath('//document[1]')[0]
    elif doc.xpath('//error') is not None:
        root = None
    else:
        root = None
        
    if root is None:
        # Root node error 
        move_file(fh, fn, "_error", tic, cik, filename, "No root or erroneous root node - moved file")
        error = True
    if error: return error

    # Check if this is an amended 10-K and throw it out if so
    type_text = root.xpath('//type/text()')
    if type_text is None or len(type_text) == 0:
        move_file(fh, fn, "_error", tic, cik, filename, "Error finding type - moved file")
        error = True
    elif type_text[0] == '10-K/A':
        move_file(fh, fn, "_amended", tic, cik, filename, "Amended 10-K - moved file")
        error = True
    if error: return error

    # Get the 'acceptance-datetime' metadata element
    acc_dt = root.xpath('//acceptance-datetime/text()')
    if acc_dt is None or len(acc_dt) == 0:
        header_text = None
        # If we didn't find an <acceptance-datetime /> element, find the date elsewhere
        if len(root.xpath('//sec-header/text()')) != 0:
            header_text = root.xpath('//sec-header/text()')[0]
        elif len(root.xpath('//ims-header/text()')) != 0:
            header_text = root.xpath('//ims-header/text()')[0]

        if header_text:
            filing_dt_text = re.sub(regex, r"\2", header_text)
        else:
            move_file(fh, fn, "_error", tic, cik, filename, "Bad filing date - moved file")
            error = True
        if error: return error
    else:
        # Get the filing date
        filing_dt_text = acc_dt[0].split('\n', 1)[0][:8]

    filing_dt = dt.strptime(filing_dt_text, '%Y%m%d')
    filing_ts = time.mktime(filing_dt.timetuple())
    begin_dt = dt(1995, 1, 1)

    # If the filing date is not within our date range, then move it
    if begin_dt > filing_dt:
        move_file(fh, fn, "_outofrange", tic, cik, filename, "Out of date range - moved file.")
        error = True
    if error: return error

    # See if we can find stock info for this company on the filing date of the 10-K
    cdef int index = 0
    cik_df = None
    try:
        index = df.index.get_loc((bytes(cik, 'utf-8'), filing_dt))
        cik_df = df.ix[bytes(cik, 'utf-8')]
        price = cik_df.ix[filing_dt, 'PRC']
        # Now, check if the price of the stock is less than $3.00
        if price < 3.0:
            move_file(fh, fn, "_nostockdata", tic, cik, filename, "Price less than $3.00 - moved file.")
            error = True
    except (IndexError, KeyError):
        # We couldn't find the cik or date for this 10-k
        move_file(fh, fn, "_nostockdata", tic, cik, filename, "No stock data found - moved file.")
        error = True
    if error: return error
    
    # Grab the report
    cdef str report = ''.join(root.xpath('//document/text')[0].itertext())

    # We will tokenize the text and iterate through each word
    cdef list tokens = report.split()
    cdef list keep_tokens = []
    cdef set stopwords_set = set(stopwords.words('english'))
    punc_table = str.maketrans("", "", string.punctuation)
    
    # Filter out words
    cdef str word
    for word in tokens:
        # Quick check to make sure we should keep filtering the word
        if len(word) != 1:
            # Strip punctuation from the word first and make it lowercase
            word = word.translate(punc_table).lower()

            # Add the word to the keep pile if it is not a stopword and if it is in 2of12inf dictionary
            if word not in stopwords_set and word in dict_2of12inf:
                keep_tokens.append(word)
            
    tokens = keep_tokens
    report = " ".join(tokens)
    cdef int total_words = len(tokens)

    # Gather info for report to save into redis
    report_hash = {
        'cik': cik,
        'tic': tic,
        'path': fn,
        'file_name': filename,
        'filing_date': filing_ts,
        'year': filing_dt.year,
        'report': report,
        'total_words': total_words,
        'company_data': pickle.dumps(cik_df),
        'index': index,
        'mtime': time.time()
    }

    # Close the file handle
    fh.close()
    
    # Save the stuff to redis
    print("Saving to redis: " + report_key)
    rds.hmset(report_key, report_hash)
    rds.set(cleaned_key, time.time())

## Process Function
The `process` function will take the cleaned file, count occurrences of positive and negative words, and calculate the historical and abnormal returns for the company based on the filing date of the 10-K. This information is stored in Redis for later access when running our analysis.

In [ ]:
%%cython
import os
import pickle
import math
import time
from datetime import datetime as dt

import pandas as pd
from pandas import DataFrame, Series

from collections import defaultdict

from __main__ import df, rds, pos_dict, pos_roots, pos_roots_map, neg_dict, neg_roots, neg_roots_map

from cpython cimport bool

def process(str fn):
    
    cdef str s = os.sep
    cdef str tic = fn.split(s)[1]
    cdef str cik = fn.split(s)[2]
    cdef str report_key = "report:" + cik + ":" + fn
    cdef str processed_key = "processed:"+ cik + ":" + fn
    
    # Get the report out of redis
    #print("Found in redis: " + report_key)
    cdef str report = str(rds.hget(report_key, 'report'))
    filing_dt = dt.fromtimestamp(int(float(rds.hget(report_key, 'filing_date').decode('utf-8'))))
    cik_df = pickle.loads(rds.hget(report_key, 'company_data'))
    cdef int index = int(rds.hget(report_key, 'index'))
    cdef dict report_hash = {}

    # Now that everything is cleaned up, we can run the word processing algorithm
    pos_occurs = defaultdict(int)
    neg_occurs = defaultdict(int)
    negators = pd.Series(['not', 'no', 'never'])

    # We will tokenize the text and iterate through each word
    tokens = pd.Series(report.split())

    # Now, process the text
    cdef int i
    cdef str token, root, word
    cdef bool negated
    for i, token in tokens.iteritems():
        if token in pos_dict:
            # Check to see if there is a negator
            negated = False
            for word in tokens.iloc[(i - 3):(i + 3)]:
                if word in negators.values:
                    #print("Found a negator: " + word + " - " + token)
                    negated = True
            if not negated:
                root = pos_roots_map[token]
                pos_occurs[root] += 1
        elif token in neg_dict:
            # Check to see if there is a negator
            negated = False
            for word in tokens.iloc[(i - 3):(i + 3)]:
                if word in negators.values:
                    #print("Found a negator: " + word + " - " + token)
                    negated = True
            if not negated:
                root = neg_roots_map[token]
                neg_occurs[root] += 1

    # For the roots we didn't find, set frequency to zero
    for root in pos_roots:
        if root not in pos_occurs:
            pos_occurs[root] = 0
    for root in neg_roots:
        if root not in neg_occurs:
            neg_occurs[root] = 0
            
    # Use the index we found earlier to grab the historical info
    hist_returns = cik_df.ix[(index + 1):, 'RET']

    # Calculate the historical return before the filing date
    cdef float hist_ret = 1.0
    for col, series in hist_returns.iteritems():
        if col == 'RET':
            for r in series:
                if not math.isnan(r):
                    hist_ret *= (r + 1.0)
    hist_ret = hist_ret - 1.0
    #print("Historical return: " + str(hist_ret))

    # Use the index we found earlier to grab the four day window returns
    returns = cik_df.ix[(index - 3):(index + 1), ['RET','vwretd']]

    # Calculate the abnormal return: r_i = M{t=0, 3} (ret_i,j) - M{t=0,3} (ret_vwi,t)
    cdef float ret = 1.0
    cdef float ret_vwi = 1.0
    for col, series in returns.iteritems():
        if col == 'RET':
            for r in series:
                if not math.isnan(r):
                    ret *= (r + 1.0)
        elif col == 'vwretd':
            for r in series:
                if  not math.isnan(r):
                    ret_vwi *= (r + 1.0)
    cdef float ab_ret = ((ret - 1.0) - (ret_vwi - 1.0))
    #print("Abnormal return: " + str(ab_ret))

    # Save results of text processing to key in redis
    report_hash['pos_occurs'] = pickle.dumps(pos_occurs)
    report_hash['neg_occurs'] = pickle.dumps(neg_occurs)
    report_hash['hist_ret'] = hist_ret
    report_hash['ab_ret'] = ab_ret
    report_hash['mtime'] = time.time()

    print("Saving to redis: " + report_key)
    rds.hmset(report_key, report_hash)
    rds.set(processed_key, time.time())

## Processing Algorithm
This cell contains the code that will go through every 10-K file that was downloaded by the scraper, check if it has been processed or cleaned, clean it, and then process it. This will take a while to run, especially if you are processing all 10-Ks from 1995 to 2008.

In [ ]:
%%cython
import os

from __main__ import check_redis, clean, process

from cpython cimport bool

# This is for testing
cdef int count = 1
cdef int stop = 100000
cdef bool skip_cleaned = True
cdef bool skip_processed = True
cdef bool process_file = True

cdef dict report_hash
cdef str fn, s, tic, cik, cleaned_key, processed_key, report_key
cdef bool cleaned, processed, error

cdef str dirpath
cdef list dirnames, filenames
cdef str folder = "SEC-Edgar-data"
for (dirpath, dirnames, filenames) in os.walk(folder, topdown=False):
    for filename in filenames:
        report_hash = {}
        fn = os.sep.join([dirpath, filename])
        
        if filename.endswith('.txt'):# and filename == "0000950116-97-000637.txt":
            if count > stop:
                break
            print(fn)
            s = os.sep
            tic = fn.split(s)[1]
            cik = fn.split(s)[2]
            
            # Check redis to see if we have processed or cleaned the report already
            cleaned_key = "cleaned:" + cik + ":" + fn
            processed_key = "processed:" + cik + ":" + fn
            report_key = "report:" + cik + ":" + fn
            (cleaned, processed) = check_redis(cleaned_key, processed_key, report_key)
            
            # If the report has been cleaned or we don't want to clean it anyway, skip this step
            error = False
            if not cleaned or not skip_cleaned:
                print("(" + str(count) + ") Cleaning " + fn)
                error = clean(fn)
                
                if not process and not error:
                    count += 1
                    continue
            if error: continue
            
            # After possibly cleaning, check if we should process the file
            if (not processed or not skip_processed) and process_file:
                print("(" + str(count) + ") Processing " + fn)
                process(fn)
                
                count += 1

## Generate Listing of Yearly Data
This cell will gather data found in Redis and generate a listing of certain information about each 10-K report for its associated year. This information about the report will be used to run the regression analysis later.

In [ ]:
count = 0
stop = math.inf
yearly_data = {}

if rds.exists("yearly-data"):
    print("Found yearly data in Redis.")
    yearly_data = pickle.loads(zlib.decompress(rds.get("yearly-data")))
else:
    keys = rds.keys("report:*")
    errors = []
    for key in keys:

        if count >= stop:
            break

        report_hash = rds.hgetall(key)
        try:
            year = 1
            cik = str(report_hash[b'cik'].decode('utf-8'))
            fn = str(report_hash[b'path'].decode('utf-8'))
            key = "report:" + cik + ":" + fn
            pos_occurs = pickle.loads(report_hash[b'pos_occurs'])
            neg_occurs = pickle.loads(report_hash[b'neg_occurs'])
            year = int(report_hash[b'year'])
            total_words = int(report_hash[b'total_words'])
            hist_ret = float(report_hash[b'hist_ret'])
            ab_ret = float(report_hash[b'ab_ret'])

            if total_words == 0:
                # Not sure why this would be zero, but we may need to reprocess
                print("Error with: " + key)
                cleaned_key = "cleaned:" + cik + ":" + fn
                processed_key = "processed:" + cik + ":" + fn

                # Delete this error from redis
                rds.delete(cleaned_key, processed_key, key)

                errors.append(key)
                continue

            try: yearly_data[year]
            except KeyError:
                yearly_data[year] = []

            year_list = yearly_data[year]
            year_list.append({
                'pos_occurs': pos_occurs,
                'neg_occurs': neg_occurs,
                'total_words': total_words,
                'hist_ret': hist_ret,
                'ab_ret': ab_ret
            })
            yearly_data[year] = year_list

            count += 1
        except KeyError:
            continue
        except e:
            print(e)
            
    print("Saving yearly data in Redis.")
    rds.set("yearly-data", zlib.compress(pickle.dumps(yearly_data)))
    
    if len(errors) > 0:
        print("Total errors: " + str(len(errors)))
        
for year in sorted(yearly_data.keys()):
    print(year, len(yearly_data[year]))

## Regression Analysis
This cell actually runs the regression analysis and will do a rolling analysis from the start year and end year. We realized too late that our CRSP and Compustat data didn't add in stock information for 1995, but this was corrected in the SAS code later. You should be able to run this analysis over the correct time frame. Also, we believe that this analysis should use 2009 as the end date in order to leverage the data from 2008 and make a prediction for the year 2009. We realized this too late and did not have time to rerun the regression analysis. These changes could affect our results and make them closer to the results from the paper.

In [ ]:
%%cython
import os
import pickle

import pandas as pd
from pandas import DataFrame, Series

import statsmodels.api as sm

from __main__ import rds, yearly_data

from cpython cimport bool

cdef int start = 1997
cdef int end = 2008
cdef bool skip_regression = True
cdef bool skip_building = True
cdef int t
cdef int year
cdef int count
cdef int total
cdef str key

# Clear some variables so we have enough RAM...
try: del data, df
except NameError:
    pass

# Generate a rolling training model using data up until year T-1
for t in range(start, (end + 1)):
    
    print("Analyzing year " + str(t))
    
    # Skip this analysis if redis already has it and the boolean is set to skip
    if rds.exists("regression:" + str(t)) and skip_regression:
        continue

    pos_word_weights = pd.DataFrame()
    neg_word_weights = pd.DataFrame()
    hist_returns = pd.DataFrame()
    ab_returns = pd.DataFrame()
    
    # Iterate over each year before year T and build the training data set
    for year in range((start - 1), t):
        
        key = "regression-data:" + str(year)
        
        if rds.exists(key) and skip_building:
            # Extract the data
            pos_word_weights = pickle.loads(rds.hget(key, 'pos_word_weights'))
            neg_word_weights = pickle.loads(rds.hget(key, 'neg_word_weights'))
            hist_returns = pickle.loads(rds.hget(key, 'hist_returns'))
            ab_returns = pickle.loads(rds.hget(key, 'ab_returns'))
            continue
        
        print("Building year " + str(year) + "/" + str(t))
        
        try: yearly_data[year]
        except KeyError:
            print("Year " + str(year) + " not found.")
            continue
        
        # Iterate through each 10-K info for the year and generate the dataframe for the regression
        count = 0
        total = len(yearly_data[year])
        for report in yearly_data[year]:
            count += 1
            print("Report " + str(count) + "/" + str(total))
            
            a = report['total_words']
            hist_ret = report['hist_ret']
            ab_ret = report['ab_ret']
            
            weights = {}
            pos_occurs = report['pos_occurs']
            for word in pos_occurs.keys():
                F = pos_occurs[word]
                weights[word] = F/(a * 1.0)
            pos_word_weights = pos_word_weights.append(weights, ignore_index=True)
            
            weights = {}
            neg_occurs = report['neg_occurs']
            for word in neg_occurs.keys():
                F = neg_occurs[word]
                weights[word] = F/(a * 1.0)
            neg_word_weights = neg_word_weights.append(weights, ignore_index=True)
            
            hist_returns = hist_returns.append({'hist_ret': hist_ret}, ignore_index=True)
            ab_returns = ab_returns.append({'ab_ret': ab_ret}, ignore_index=True)
            
        # Save our progress so we don't have to recalculate this year again
        print("Saving progress for year " + str(year) + " to redis.")
        reg_data_hash = {
            'pos_word_weights': pickle.dumps(pos_word_weights),
            'neg_word_weights': pickle.dumps(neg_word_weights),
            'hist_returns': pickle.dumps(hist_returns),
            'ab_returns': pickle.dumps(ab_returns)
        }
        rds.hmset(key, reg_data_hash)
        
    # Run the regressions for all years up to year T
    if not ab_returns.empty and not hist_returns.empty and not pos_word_weights.empty and not neg_word_weights.empty:
        hist_returns.reset_index()
        hist_returns_series = pd.Series(hist_returns['hist_ret'])
        ab_returns.reset_index()
        ab_returns_series = pd.Series(ab_returns['ab_ret'])
        pos_word_weights.reset_index()
        neg_word_weights.reset_index()
        
        # Estimate the weights for the words using a regression
        print("T = " + str(t) + ": Estimating weights")
        pos_reg = sm.OLS(hist_returns_series, pos_word_weights)
        pos_model = pos_reg.fit()
        neg_reg = sm.OLS(hist_returns_series, neg_word_weights)
        neg_model = neg_reg.fit()
        
        # Map the words to their coefficients
        pos_coeffs_dict = dict(zip(list(pos_word_weights.columns), pos_model.params))
        pos_coeffs = pd.DataFrame(list(pos_coeffs_dict.items()), columns=['word','weight'])
        neg_coeffs_dict = dict(zip(list(neg_word_weights.columns), neg_model.params))
        neg_coeffs = pd.DataFrame(list(neg_coeffs_dict.items()), columns=['word','weight'])
    
        # Calculate the average word weight as well as the standard deviation
        pos_avg = pos_coeffs['weight'].mean()
        pos_std = pos_coeffs['weight'].std()
        neg_avg = neg_coeffs['weight'].mean()
        neg_std = neg_coeffs['weight'].std()

        # Normalize the weights of the words
        print("T = " + str(t) + ": Normalizing weights")
        pos_norm = list()
        for col, series in pos_coeffs.iteritems():
            if col == 'weight':
                for weight in series:
                    pos_norm.append((weight - pos_avg) / pos_std)
        pos_coeffs['norm_weight'] = pd.Series(pos_norm, index=pos_coeffs.index)
        
        neg_norm = list()
        for col, series in neg_coeffs.iteritems():
            if col == 'weight':
                for weight in series:
                    neg_norm.append((weight - neg_avg) / neg_std)
        neg_coeffs['norm_weight'] = pd.Series(neg_norm, index=neg_coeffs.index)
        
        # Iterate through the original word weights and apply the normalized weight
        for word, series in pos_word_weights.iteritems():
            norm_weight = pos_coeffs.loc[pos_coeffs['word'] == word]['norm_weight']
            pos_word_weights[word] = series.apply(lambda x: x * norm_weight)
        for word, series in neg_word_weights.iteritems():
            norm_weight = neg_coeffs.loc[neg_coeffs['word'] == word]['norm_weight']
            neg_word_weights[word] = series.apply(lambda x: x * norm_weight)
                
        # Run the regression for abnormal (after filing) returns using the estimated weights for the words
        print("T = " + str(t) + ": Doing regression")
        pos_ab_reg = sm.OLS(ab_returns_series, pos_word_weights)
        pos_ab_model = pos_ab_reg.fit()
        neg_ab_reg = sm.OLS(ab_returns_series, neg_word_weights)
        neg_ab_model = neg_ab_reg.fit()
        
        # Map the words to their coefficients
        pos_coeffs_dict = dict(zip(list(pos_word_weights.columns), pos_ab_model.params))
        pos_coeffs = pd.DataFrame(list(pos_coeffs_dict.items()), columns=['word','weight'])
        neg_coeffs_dict = dict(zip(list(neg_word_weights.columns), neg_ab_model.params))
        neg_coeffs = pd.DataFrame(list(neg_coeffs_dict.items()), columns=['word','weight'])
        
        # Store the regression models in Redis
        key = "regression:" + str(t)        
        reg_hash = {
            'pos_model': pickle.dumps(pos_model),
            'neg_model': pickle.dumps(neg_model),
            'pos_ab_model': pickle.dumps(pos_ab_model),
            'neg_ab_model': pickle.dumps(neg_ab_model)
        }        
        rds.hmset(key, reg_hash)


## Reports
This cell contains the code that will run two reports. The first report is a "matrix" of both positive and negative term frequency grouped by their weight frequency. This is essentially our version of `Table II` from the Appendix of the *Word Power* paper. The second report is a listing of both positive and negative terms group by their term frequency (least frequently occurring to most frequently) and the top five words within those buckets are the ones with the highest coefficients from the regression analysis. Therefore, this is essentially listing out the most effective words (based on returns) grouped by their frequency.

In [ ]:
# This will store the global positive and negative words occurrances
pos_occurs_all = defaultdict(int)
neg_occurs_all = defaultdict(int)

try: yearly_data
except NameError: yearly_data = pickle.loads(zlib.decompress(rds.get('yearly-data')))

if rds.exists("freq:pos:all") and rds.exists("freq:neg:all"):
    pos_occurs_all = pickle.loads(zlib.decompress(rds.get("freq:pos:all")))
    neg_occurs_all = pickle.loads(zlib.decompress(rds.get("freq:neg:all")))
else:
    for year in yearly_data:
        print("Counting frequency for year " + str(year))

        pos_occurs_year = defaultdict(int)
        neg_occurs_year = defaultdict(int)
        pos_key = "freq:pos:" + str(year)
        neg_key = "freq:neg:" + str(year)

        if rds.exists(pos_key) and rds.exists(neg_key):
            pos_occurs_year = pickle.loads(zlib.decompress(rds.get(pos_key)))
            neg_occurs_year = pickle.loads(zlib.decompress(rds.get(neg_key)))
        else:
            count = 0
            total = len(yearly_data[year])
            for report in yearly_data[year]:
                count += 1
                print("Counting Report " + str(count) + "/" + str(total))
                report_pos_occurs = report['pos_occurs']
                report_neg_occurs = report['neg_occurs']

                for word, freq in report_pos_occurs.items():
                    pos_occurs_year[word] += freq
                for word, freq in report_neg_occurs.items():
                    neg_occurs_year[word] += freq

            print("Saving progress to Redis.")
            rds.set("freq:pos:" + str(year), zlib.compress(pickle.dumps(pos_occurs_year)))
            rds.set("freq:neg:" + str(year), zlib.compress(pickle.dumps(neg_occurs_year)))

    for word, freq in pos_occurs_year.items():
        pos_occurs_all[word] += freq
    for word, freq in neg_occurs_year.items():
        neg_occurs_all[word] += freq
        
    print("Finished totaling and now saving to Redis.")
    rds.set("freq:pos:all", zlib.compress(pickle.dumps(pos_occurs_all)))
    rds.set("freq:neg:all", zlib.compress(pickle.dumps(neg_occurs_all)))

# Sort the values by highest to lowest
pos_sorted = pd.Series(data=pos_occurs_all).sort_values()
neg_sorted = pd.Series(data=neg_occurs_all).sort_values()

# Get the coefficients
pos_coeffs = pickle.loads(zlib.decompress(rds.hget("regression:2008", 'pos_coeffs')))
neg_coeffs = pickle.loads(zlib.decompress(rds.hget("regression:2008", 'neg_coeffs')))

# Place the weights into quintiles
pos_wt_buckets = pd.qcut(pos_coeffs['weight'], 5, labels=range(1,6))
neg_wt_buckets = pd.qcut(neg_coeffs['weight'], 5, labels=range(1,6))

# Sort the coefficients by value
pos_coeffs.sort_values('weight', inplace=True)
neg_coeffs.sort_values('weight', inplace=True)

# See if we have the total 10-Ks and number of appearances of each word in a 10-k
try: total, pos_appearances, neg_appearances
except NameError:
    (total, pos_appearances, neg_appearances) = get_appearances()

# Place the frequencies into quintiles
pos_freq_buckets = pd.qcut(pos_appearances['apps'], 5, labels=range(1,6))
neg_freq_buckets = pd.qcut(neg_appearances['apps'], 5, labels=range(1,6))

bucket_freq_list = []
bucket_wt_list = []
freq_list = []
total_list = []
for (word, row) in pos_coeffs.iterrows():
    bucket_freq_list.append(pos_freq_buckets.ix[word])
    bucket_wt_list.append(pos_wt_buckets.ix[word])
    freq_list.append(pos_sorted.ix[word])
    total_list.append(pos_appearances.ix[word]['apps'])
pos_coeffs['bucket_freq'] = pd.Series(bucket_freq_list, index=pos_coeffs.index)
pos_coeffs['bucket_weight'] = pd.Series(bucket_wt_list, index=pos_coeffs.index)
pos_coeffs['freq'] = pd.Series(freq_list, index=pos_coeffs.index)
pos_coeffs['apps'] = pd.Series(total_list, index=pos_coeffs.index)
pos_coeffs['files'] = pos_appearances['files']

bucket_freq_list = []
bucket_wt_list = []
freq_list = []
total_list = []
for (word, row) in neg_coeffs.iterrows():
    bucket_freq_list.append(neg_freq_buckets.ix[word])
    bucket_wt_list.append(neg_wt_buckets.ix[word])
    freq_list.append(neg_sorted.ix[word])
    total_list.append(neg_appearances.ix[word]['apps'])
neg_coeffs['bucket_freq'] = pd.Series(bucket_freq_list, index=neg_coeffs.index)
neg_coeffs['bucket_weight'] = pd.Series(bucket_wt_list, index=neg_coeffs.index)
neg_coeffs['freq'] = pd.Series(freq_list, index=neg_coeffs.index)
neg_coeffs['apps'] = pd.Series(total_list, index=neg_coeffs.index)
neg_coeffs['files'] = neg_appearances['files']

# Build the table of weight quintiles to frequency quintiles
pos_tab5 = pos_coeffs.reset_index(level='word').set_index(['bucket_weight','bucket_freq','word'])
neg_tab5 = neg_coeffs.reset_index(level='word').set_index(['bucket_weight','bucket_freq','word'])

# Iterate first through the weight buckets
print("Positive Weight Quintiles by Term Frequency Quintiles")
for i in range(1,6):
    # Get the total number of words in this weight quantile
    tot_in_quantile = len(pos_tab5.ix[i]['weight'])
    
    print("\nWeight Quintile " + str(i) + ": Total words = " + str(tot_in_quantile))
    
    # Find the number of words in each freq bucket and show that as percentage of words in weight bucket
    for j in range(1,6):
        num_words = len(pos_tab5.ix[(i,j)]['weight'])
        perc = (num_words / (1.0 * tot_in_quantile)) * 100
        print("Term Freq Quintile " + str(j) + ": " + str(perc))
        
print("\nNegative Weight Quintiles by Term Frequency Quintiles")
for i in range(1,6):
    tot_in_quantile = len(neg_tab5.ix[i]['weight'])
    
    print("\nWeight Quintile " + str(i) + ": Total words = " + str(tot_in_quantile))
    
    for j in range(1,6):
        num_words = len(neg_tab5.ix[(i,j)]['weight'])
        perc = (num_words / (1.0 * tot_in_quantile)) * 100
        print("Term Freq Quintile " + str(j) + ": " + str(perc))


# Now, sort by the frequency buckets (ascending) and weights (descending)
pos_coeffs.sort_values(['bucket_freq','weight'], ascending=[True,False], inplace=True)
neg_coeffs.sort_values(['bucket_freq','weight'], ascending=[True,False], inplace=True)

# Get the top 5 words in each quintile
pos_top5 = pos_coeffs.reset_index(level=['word']).set_index(['bucket_freq'])
pos_top5_list = {}
neg_top5 = neg_coeffs.reset_index(level=['word']).set_index(['bucket_freq'])
neg_top5_list = {}
for i in range(1, 6):
    pos_words = []
    pos_bucket_words = pos_top5.ix[i]['word']
    neg_words = []
    neg_bucket_words = neg_top5.ix[i]['word']
    
    count = 0
    for word in pos_bucket_words:
        if count >= 5:
            break
        pos_words.append(word)
        count += 1
    count = 0
    for word in neg_bucket_words:
        if count >= 5:
            break
        neg_words.append(word)
        count += 1
    
    pos_top5_list[i] = pos_words
    neg_top5_list[i] = neg_words

# Print out most frequent positive words
header = "Top Five Most Positive and Negative Words within Frequency Quintiles"
print(header)
print("="*len(header))

subheader = "\nTop 5 Most Positive Words"
print(subheader)
print("="*len(subheader))

for (bucket, words) in pos_top5_list.items():
    print("\nBucket " + str(bucket))
    for word in words:
        print(word)

# Print out most frequent negative words
subheader = "\nTop 5 Most Negative Words"
print(subheader)
print("="*len(subheader))

for (bucket, words) in neg_top5_list.items():
    print("\nBucket " + str(bucket))
    for word in words:
        print(word)

## Get Appearances Function
This cell contains the `get_appearances` function which is used above in generating the reports that show which words occur the most frequently. This function will count the number of appearances for each positive or negative word across all 10-Ks we are analyzing. Note that it will calculate this based on the total number of 10-Ks in which the word appears divided by the total number of 10-Ks that we processed in order to calculate the total percentage of 10-Ks in which the word appears.

In [ ]:
# This function will get a count of the number of appearances for each positive or negative word across all 10-Ks.
def get_appearances():
    try: yearly_data
    except NameError: yearly_data = pickle.loads(zlib.decompress(rds.get("yearly-data")))

    pos_appearances_dict = defaultdict(int)
    neg_appearances_dict = defaultdict(int)
    pos_app_set = {}
    neg_app_set = {}    

    total = 0
    for year in yearly_data:
        total += len(yearly_data[year])

        i = 0
        for report in yearly_data[year]:
            # Keep track if a word appears at least once and increment a counter if so
            pos_occurs = report['pos_occurs']
            neg_occurs = report['neg_occurs']

            for (word, freq) in pos_occurs.items():
                try: word_set = pos_app_set[word]
                except: word_set = set()
                    
                if freq > 0:
                    pos_appearances_dict[word] += 1
                    word_set.add(str(year) + ":" + str(i))
                    pos_app_set[word] = word_set
                else:
                    pos_appearances_dict[word] += 0
                    pos_app_set[word] = word_set
                    
            for (word, freq) in neg_occurs.items():
                if freq > 0:
                    neg_appearances_dict[word] += 1
                    word_set.add(str(year) + ":" + str(i))
                    neg_app_set[word] = word_set
                else:
                    neg_appearances_dict[word] += 0
                    neg_app_set[word] = word_set
            i += 1

    pos_appearances = pd.DataFrame({'apps': pos_appearances_dict, 'files': pos_app_set})
    neg_appearances = pd.DataFrame({'apps': neg_appearances_dict, 'files': neg_app_set})
    return (total, pos_appearances, neg_appearances)